In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-23
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-23.parquet
dest_path: ..\data\2025-04-23_df_OHLCV_clean.parquet


In [3]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open    High     Low   Close  Adj Close   Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                       
RY     2025-04-23  119.27  119.86  117.42  117.79     117.79   976958    119.27    119.86   117.42
       2025-04-22  115.82  117.97  115.41  117.97     117.97  5486100    115.82    117.97   115.41
       2025-04-21  116.00  116.60  114.63  115.21     115.21   600100    116.00    116.60   114.63
       2025-04-17  115.53  116.71  115.45  116.24     116.24  1090900    115.53    116.71   115.45
       2025-04-16  115.21  116.24  114.25  115.62     115.62   935000    115.21    116.24   114.25

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 387583 entries, ('RY', Timestamp('2025-04-23 00:00:00')) to ('TPR', Timestamp('2024-04-24 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       387583 non-null  float64
 1   High       387583 non-null  float64
 2   Low        387583 non-null  float64
 3   Close      387583 non-null  float64
 4   Adj Close  387583 non-null  float64
 5   Volume     387569 non-null  Int64  
 6   Adj Open   387583 non-null  float64
 7   Adj High   387583 non-null  float64
 8   Adj Low    387583 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.5+ MB


In [4]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

Original number of symbols: 1557
Number of symbols after filtering: 1540
Number of symbols filtered out: 17

First 10 symbols that were filtered out:
['LTM', 'SARO', 'VIK', 'TEM', 'VG', 'LOAR', 'RBRK', 'SAIL', 'WAY', 'TTAN']

Example of dates for first filtered out symbol:

Dates for LTM:
DatetimeIndex(['2025-04-23', '2025-04-22', '2025-04-21', '2025-04-17', '2025-04-16', '2025-04-15', '2025-04-14', '2025-04-11', '2025-04-10', '2025-04-09',
               ...
               '2024-08-07', '2024-08-06', '2024-08-05', '2024-08-02', '2024-08-01', '2024-07-31', '2024-07-30', '2024-07-29', '2024-07-26', '2024-07-25'], dtype='datetime64[ns]', name='Date', length=187, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 385000 entries, ('RY', Timestamp('2025-04-23 00:00:00')) to ('TPR', Timestamp('2024-04-24 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       385000 non-n


[Cleaning Report]
Removed 3 symbols with missing data: ['FSEC', 'BULL', 'FER']

[Cleaned Data Structure]
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384250 entries, ('RY', Timestamp('2025-04-23 00:00:00')) to ('TPR', Timestamp('2024-04-24 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       384250 non-null  float64
 1   High       384250 non-null  float64
 2   Low        384250 non-null  float64
 3   Close      384250 non-null  float64
 4   Adj Close  384250 non-null  float64
 5   Volume     384250 non-null  Int64  
 6   Adj Open   384250 non-null  float64
 7   Adj High   384250 non-null  float64
 8   Adj Low    384250 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.3+ MB


In [5]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\2025-04-23_df_OHLCV_clean.parquet
